In [ ]:
import os

import pandas as pd
import seaborn as sns

from ray import init, rllib, tune, shutdown

In [ ]:
from attack_simulator.agents import ATTACKERS
from attack_simulator.env import AttackSimulationEnv
from attack_simulator.graph import AttackGraph, SIZES

In [ ]:
class NoAction(rllib.policy.Policy):
    def compute_actions(self, observations, *args, **kwargs):
        return [0 for _ in observations], [], {}

    def get_weights(self):
        return {}

    def set_weights(self, weights):
        pass
    
    def learn_on_batch(self, samples):
        return {}

    
no_action = rllib.agents.trainer_template.build_trainer(name="NoAction", default_policy=NoAction)

In [ ]:
from ray.util.client import worker

worker.INITIAL_TIMEOUT_SEC = worker.MAX_TIMEOUT_SEC = 1


def ray_init():
    if os.path.isdir("/var/run/secrets/kubernetes.io") or os.path.exists(
        os.path.expanduser("~/ray_bootstrap_config.yaml")
    ):
        # inside k8s pod or a cluster managed by Ray's autoscaler
        context = init(address="auto")
    else:
        ray_client_server = "host.docker.internal" if os.path.exists("/.dockerenv") else "127.0.0.1"
        try:
            context = init(address=f"ray://{ray_client_server}:10001")
        except ConnectionError:
            # clean up after failed connection attempt
            shutdown()
            # listen on all interfaces inside a container for port-forwarding to work
            dashboard_host = "0.0.0.0" if os.path.exists("/.dockerenv") else "127.0.0.1"
            context = init(num_cpus=4, dashboard_host=dashboard_host)
    print("\x1b[33;1m", context, "\x1b[m")
    return context

In [ ]:
num_episodes = 5  # run 5 episodes on the same environment
attackers = list(ATTACKERS)
graphs = [AttackGraph(dict(graph_size=size)) for size in SIZES]

config = dict(
    framework="torch",
    env=AttackSimulationEnv,
    env_config=dict(attack_graph=tune.grid_search(graphs), attacker=tune.grid_search(attackers)),
    seed=tune.grid_search([0, 1, 2, 3, 6, 7, 11, 28, 42, 1337]),
    # evaluation ONLY: avoid MultiGPU optimizer, set all relevant sizes to 0
    simple_optimizer=True,
    num_workers=0,
    train_batch_size=0,
    rollout_fragment_length=0,
    timesteps_per_iteration=0,
    batch_mode="complete_episodes",
    # define evaluation settings
    evaluation_interval=1,
    evaluation_num_workers=1,
    evaluation_config=dict(explore=False),
    evaluation_num_episodes=num_episodes,
)

In [ ]:
rename = {
    "config.env_config.attacker": "Attacker",
    "config.env_config.attack_graph": "graph",
    "evaluation.hist_stats.episode_reward": "returns",
    "evaluation.hist_stats.episode_lengths": "lengths",
}


def postprocess(results_df):
    df = results_df[rename.keys()].rename(columns=rename)
    df.dropna(inplace=True)  # remove `NaN` results from failed trials
    df["Graph size"] = df["graph"].apply(lambda g: g.num_attacks)
    del df["graph"]
    df["tuple"] = df.apply(lambda t: list(zip(t.returns, t.lengths)), axis="columns")
    del df["returns"]
    del df["lengths"]
    df = df.explode("tuple", ignore_index=True)
    df[["Returns", "Episode lengths"]] = df["tuple"].tolist()
    del df["tuple"]
    return df

In [ ]:
def generate(savename):
    ray_init()
    results = tune.run(
        no_action,
        config=config,
        stop=dict(training_iteration=0),
        max_failures=3,
        queue_trials=True,
        raise_on_failed_trial=False,
        progress_reporter=tune.JupyterNotebookReporter(overwrite=True),
        sync_config=tune.SyncConfig(sync_to_driver=False),
    )
    shutdown()
    df = postprocess(results.results_df)
    df.to_csv(savename)
    return df

In [ ]:
savename = "length-tune-eval.csv"

df = generate(savename) if not os.path.exists(savename) else pd.read_csv(savename, index_col=0)

In [ ]:
sns.set(style="darkgrid", rc={"figure.figsize": (12, 8)})

In [ ]:
g = sns.lineplot(data=df, x="Graph size", y="Returns", hue="Attacker", ci="sd")
g.legend(title="Attacker", loc="lower left")
g.set_title("Defender: no-action")

In [ ]:
g = sns.lineplot(data=df, x="Graph size", y="Episode lengths", hue="Attacker", ci="sd")
g.legend(title="Attacker", loc="upper left")
g.set_title("Defender: no-action")

In [ ]:
pd.set_option("display.max_columns", 32)
df.groupby("Attacker").describe()